## Slice the band mapping or band structure data along certain high-symmetry lines
The following demo relates to the software package [mpes](https://github.com/mpes-kit/mpes/), which contains the functionality for making band path figures

In [ ]:
import warnings as wn
wn.filterwarnings("ignore")

import numpy as np
import fuller
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpes import analysis as aly, fprocessing as fp
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline

In [ ]:
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Arial'
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [ ]:
# Load photoemission data files
fdir = r'../../data/pes'
files = fuller.utils.findFiles(fdir, fstring='/*', ftype='h5')
fdata = fp.readBinnedhdf5(files[1])
mc = aly.MomentumCorrector(fdata['V'])

In [ ]:
mc.selectSlice2D(selector=slice(30, 32), axis=2)
mc.featureExtract(mc.slice, method='daofind', sigma=6, fwhm=20, symscores=False)

# False detection filter, if needed
try:
    mc.pouter_ord = mc.pouter_ord[[0,1,3,5,6,9],:]
except:
    pass

In [ ]:
mc.view(image=mc.slice, annotated=True, points=mc.features)

In [ ]:
# Define high-symmetry points
G = mc.pcent # Gamma point
K = mc.pouter_ord[0,:] # K point
K1 = mc.pouter_ord[1,:] # K' point
M = (K + K1) / 2 # M point

# Define cutting path
pathPoints = np.asarray([G, M, K, G])
nGM, nMK, nKG = 70, 39, 79
segPoints = [nGM, nMK, nKG]
rowInds, colInds, pathInds = aly.points2path(pathPoints[:,0], pathPoints[:,1], npoints=segPoints)
nSegPoints = len(rowInds)

# Normalize data
imNorm = fdata['V'] / fdata['V'].max()

# Sample the data along high-symmetry lines (k-path) connecting the corresponding high-symmetry points
pathDiagram = aly.bandpath_map(imNorm, pathr=rowInds, pathc=colInds, eaxis=2)

In [ ]:
Evals = fdata['E']
ehi, elo = Evals[0], Evals[469]

f, ax = plt.subplots(figsize=(10, 6))
plt.imshow(pathDiagram[:470, :], cmap='Blues', aspect=10.9, extent=[0, nSegPoints, elo, ehi], vmin=0, vmax=0.5)
ax.set_xticks(pathInds)
ax.set_xticklabels(['$\overline{\Gamma}$', '$\overline{\mathrm{M}}$',
                    '$\overline{\mathrm{K}}$', '$\overline{\Gamma}$'], fontsize=15)
for p in pathInds[:-1]:
    ax.axvline(x=p, c='r', ls='--', lw=2, dashes=[4, 2])
# ax.axhline(y=0, ls='--', color='r', lw=2)
ax.yaxis.set_major_locator(MultipleLocator(2))
ax.yaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
ax.set_ylabel('Energy (eV)', fontsize=15, rotation=-90, labelpad=20)
ax.tick_params(axis='x', length=0, pad=6)
ax.tick_params(which='both', axis='y', length=8, width=2, labelsize=15)